SPDX-License-Identifier: Apache-2.0
Copyright (c) 2023, Rahul Unnikrishnan Nair <rahul.unnikrishnan.nair@intel.com>

In [2]:
# if not installed, install the bigDL library for xpus:
#pip install --pre --upgrade bigdl-llm[xpu]==2.4.0b20231028 -f https://developer.intel.com/ipex-whl-stable-xpu

# Streamlining Text-to-SQL Tasks: Fine-Tuning LLMs with Quantized LoRA on Intel

## Applying Quantized LoRA and Hugging Face Transformers on Intel Architecture

This Jupyter Notebook is designed to facilitate the fine-tuning of Language Models for specialized tasks such as Text-to-SQL conversions. The notebook is suitable for AI engineers and practitioners looking to leverage the power of Intel hardware for fine-tuning models efficiently.tuning of Language Models for specialized tasks such as Text-to-SQL conversions. The notebook is suitable for AI engineers and practioners looking to leverage the power of Intel hardware for fine-tuning models efficiently.

### What you will learn with this Notebook
- Fine-tune a Language Model with either a pre-existing dataset or a custom dataset tailored to your needs.
- Gain insights into the fine-tuning process, including how to manipulate various training parameters to optimize your model's performance.
- Test different configurations and observe the results in real-time.

### Hardware Compatibility
- The notebook is compatible with 4th Generation Intel® Xeon® Scalable Processors, ensuring high performance for AI-related tasks.
- Optimizations are in place for the Intel® Data Center GPU Max Series, offering advanced AI acceleration capabilities.

### Fine-Tuning Methodology
We use the Quantized Low Rank Adapter (QLoRA) method for fine-tuning, which allows for a foundational model to be enhanced with task-specific adapters efficiently. This method is chosen for its computational efficiency and the flexibility it provides in quickly adapting to different tasks, like Text-to-SQL translation.

LoRA (Low-Rank Adaptation) works by inserting small trainable adapter modules between the layers of a large pretrained model like an LLM.These adapters are low-rank matrices of shape (r x d) where r is a small rank hyperparameter and d is the hidden dimension size. For example, r may be 64 while d is 1024 or larger.The low-rank structure comes from factorizing the adapter matrix into two smaller matrices L1 and L2 of shape (r x d) and (d x r). The adapter then computes L1 * L2. This factorization allows the adapter to have far fewer trainable parameters compared to the full d x d dimensions, yet still adapt the function of each layer.

LoRA adapters are scaled by a parameter α during training to control their capacity. Dropout is also added to regularize them. During finetuning, only the adapter parameters L1 and L2 are updated, while the original pretrained model weights remain fixed. This allows efficiently optimizing just the small adapters rather than all the parameters.

QLoRA builds on top of LoRA by first quantizing the pretrained model weights before adding adapters. For example, the weights may be quantized to 4 bits rather than 32-bit floats. This quantization shrinks the model size in memory, providing further savings. QLoRA keeps the LoRA adapters in full precision for accuracy. During training, the quantized weights are temporarily dequantized on-the-fly to compute the forward and backward passes. But only the adapter gradients are accumulated, not the base model.

For a deeper understanding of LoRA, readers are encouraged to refer to the [paper](https://arxiv.org/abs/2106.09685), "LoRA: Low-Rank Adaptation of Large Language Models", which details the methodology and its application to models as large as GPT-3. The subsequent development, QLoRA, extends these principles with quantization techniques as detailed in the [paper](https://arxiv.org/abs/2305.14314/) "Quantized LoRA: Scaling Up Low-Rank Adaptation to Large Language Models".

In summary, LoRA uses low-rank adapters to minimize new parameters, while QLoRA adds quantization of the base model for further memory savings. Together they enable highly efficient finetuning. With these optimizations, this notebook demonstrates how to fine-tune a language model on Intel architecture using Hugging Face Transformers and the [BigDL](https://github.com/intel-analytics/BigDL) library for efficient Text-to-SQL task adaptation.

Let's begin our journey into fine-tuning a model that can adeptly translate natural language queries into SQL statements.

## Initialization
Import all the necessary libraries required for the fine-tuning process.

In [7]:
!pip install peft --no-deps

Defaulting to user installation because normal site-packages is not writeable
  Using cached peft-0.5.0-py3-none-any.whl (85 kB)


In [9]:
import logging
import os
import sys
from math import ceil
from typing import Optional, Tuple
import warnings

warnings.filterwarnings(
    "ignore", category=UserWarning, module="intel_extension_for_pytorch"
)
warnings.filterwarnings(
    "ignore", category=UserWarning, module="torchvision.io.image", lineno=13
)
warnings.filterwarnings('ignore', message='You are using the default legacy behaviour')
warnings.filterwarnings('ignore', category=UserWarning, message='.*Parameter.*')
warnings.filterwarnings('ignore', category=FutureWarning, message='This implementation of AdamW is deprecated')
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("bigdl").setLevel(logging.ERROR)


import torch
import intel_extension_for_pytorch as ipex
from datasets import load_dataset
from datasets import Dataset
from bigdl.llm.transformers import AutoModelForCausalLM
from bigdl.llm.transformers.qlora import (
    get_peft_model,
    prepare_model_for_kbit_training as prepare_model,
)
from peft import LoraConfig
import transformers
from transformers import (
    DataCollatorForSeq2Seq,
    LlamaTokenizer,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)
transformers.logging.set_verbosity_error()

ImportError: cannot import name 'is_torch_xpu_available' from 'transformers.utils' (/home/ue4799802b0f9e4bc38f5f2b87003ee2/.local/lib/python3.9/site-packages/transformers/utils/__init__.py)

## Configuration

Set up basic configurations for fine-tuning. These include the base model to use, data paths, and device settings.

In [2]:
# TODO(rahul): Move these to a config file later
BASE_MODEL = "openlm-research/open_llama_3b_v2"
DATA_PATH = "b-mc2/sql-create-context"
MODEL_PATH = "./final_model"
ADAPTER_PATH = "./lora_adapters"
DEVICE = torch.device("xpu" if torch.xpu.is_available() else "cpu")
LORA_CONFIG = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

## Prompt Engineering for Text-to-SQL Conversion

In the realm of fine-tuning language models for specialized tasks, the design of the prompt is a pivotal aspect. The function `generate_prompt_sql` plays a crucial role in this process, particularly for the task of converting natural language questions into SQL queries.

### The Importance of a Well-Defined Prompt

A well-defined prompt is instrumental for several reasons:

- **Clarity of Task**: It communicates to the model the exact nature of the task it is expected to perform. In this case, the model is directed to generate SQL queries based on given questions and contextual information about the database.

- **Consistent Format**: Consistency in the prompt structure allows for uniform training examples. This helps the model to understand and learn the pattern of the input-to-output relationship, which is vital for generating correct SQL queries.

- **Inclusion of Context**: The function ensures that the model is provided with the database context or schema, a critical piece of information required to formulate valid SQL statements.

- **Result Conditioning**: By incorporating an optional expected output, the model can be fine-tuned more effectively, guiding it towards the desired output format.

### Crafting the Prompt

The `generate_prompt_sql` function is crafted to encapsulate the input question, the relevant database context, and the expected output in a structured and concise manner. This structure is not just a facilitator for the model's learning process but also serves as a debugging tool, allowing for easier inspection of how the model processes and responds to the input.

With this function, we are not just fine-tuning a model; we are engineering a precise and efficient interaction between the user's natural language queries and the model's SQL generation capabilities. This forms the bedrock of a reliable text-to-SQL conversion system, tailored to deliver accurate and useful SQL queries in response to diverse and complex questions.


In [3]:
def generate_prompt_sql(input_question, context, output=""):
    """
    Generates a prompt for fine-tuning the LLM model for text-to-SQL tasks.

    Parameters:
        input_question (str): The input text or question to be converted to SQL.
        context (str): The schema or context in which the SQL query operates.
        output (str, optional): The expected SQL query as the output.

    Returns:
        str: A formatted string serving as the prompt for the fine-tuning task.
    """
    return f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. 

You must output the SQL query that answers the question.

### Input:
{input_question}

### Context:
{context}

### Response:
{output}"""

## FineTuner

The `FineTuner` class encapsulates the entire process of fine-tuning large language models (LLMs) for specialized tasks such as text-to-SQL conversion. Its design adheres to best practices in machine learning and leverages the powerful features of pre-trained models, quantization techniques, and advanced tokenization processes.

## Model Loading and Configuration
When initializing the `FineTuner`, we load the base model specified by the `base_model_id`. Crucially, we use the `load_in_low_bit` parameter with the value "nf4", which allows the model to load in a 4-bit format, significantly reducing memory footprint while maintaining performance. The `optimize_model` flag is set to `False` to ensure that the original architecture of the model is preserved.

The model is also set to use a `torch_dtype` of `torch.float16`, enabling mixed-precision training for faster computation and lower memory usage. By specifying `modules_to_not_convert`, we can control which parts of the model are kept in full precision, which is critical for maintaining certain functionalities, like the `lm_head`, at maximum fidelity.

## Tokenization Strategy
The tokenization process is tailored to the type of model being fine-tuned. For instance, if we are working with a Llama model, we utilize a `LlamaTokenizer` to ensure compatibility with the model's expected input format. For other models, a generic `AutoTokenizer` is used.

We configure the tokenizer to pad from the left side (`padding_side="left"`) and set the pad token ID to 0, which is a common practice for certain language models that are sensitive to the position and order of tokens.

## Data Tokenization and Preparation
The `tokenize_batch` method is where the fine-tuner ingests raw text data and converts it into a format suitable for training the model. This method handles the addition of end-of-sequence tokens, truncation to a specified `cutoff_len`, and conditioning on the input for training.

## Dataset Handling
`prepare_data` manages the splitting of data into training and validation sets, applying the `tokenize_batch` transformation to each entry. This ensures that our datasets are ready for input into the model, with all necessary tokenization applied.

## Training Process
Finally, the `train_model` method orchestrates the training process, setting up the `Trainer` with the correct datasets, training arguments, and data collator. It ensures that caching is disabled (`use_cache = False`) for the model during training, which can be an essential step for some LLMs that have large memory footprints.

The fine-tuning process is encapsulated within the `finetune` method, which strings together all the previous steps into a coherent pipeline, from model setup to training execution.

By abstracting the fine-tuning process into a class with clear methods for each step, we enable a modular and understandable approach to enhancing LLMs for specific tasks. This modularity not only makes the codebase maintainable but also provides clear points for customization and optimization for different hardware setups, such as Intel® XPU.

The key steps to using QLoRA for efficient finetuning are:

- Load a pretrained model like LLaMA and initialize it in low precision mode by setting load_in_low_bit="nf4". This will load the model with weights quantized to 4-bit NormalFloat (nf4).
- Prepare the quantized model for finetuning with prepare_model(model). This handles quantizing the model weights into blocks and computing quantization constants.
- Add LoRA adapters to the model using get_peft_model(model, config). The config defines the LoRA hyperparameters like adapter size, dropout, etc.
- Finetune the model by passing gradients only through the adapters. The base model weights remain fixed in low precision.

Looking at the code:

- AutoModelForCausalLM is loaded with load_in_low_bit="nf4" to initialize in 4-bit.
- The model is prepared via prepare_model() which quantizes weights into blocks.
- LoRA adapters are added with get_peft_model() using the provided config.
- Trainer finetunes the model, with gradients only flowing into the adapters.

So in summary, we leverage QLoRA in BigDL to load the base LLM in low precision, inject adapters, and efficiently finetune by optimizing just the adapters end-to-end while keeping the base model fixed. This unlocks huge memory savings, allowing us to adapt giant models on ordinary GPUs.


In [4]:
class FineTuner:
    """A class to handle the fine-tuning of LLM models."""

    def __init__(self, base_model_id: str, model_path: str, device: torch.device):
        """
        Initialize the FineTuner with base model, model path, and device.

        Parameters:
            base_model_id (str): Id of pre-trained model to use for fine-tuning.
            model_path (str): Path to save the fine-tuned model.
            device (torch.device): Device to run the model on.
        """
        self.base_model_id = base_model_id
        self.model_path = model_path
        self.device = device

    def setup_models(self):
        """Downloads the pre-trained model and tokenizer based on the given base model ID."""
        try:
            self.model = AutoModelForCausalLM.from_pretrained(
                self.base_model_id,
                load_in_low_bit="nf4",
                optimize_model=False,
                torch_dtype=torch.float16,
                modules_to_not_convert=["lm_head"],
            )
            # Choose the appropriate tokenizer based on the model name
            if 'llama' in self.base_model_id.lower():
                self.tokenizer = LlamaTokenizer.from_pretrained(self.base_model_id)
            else:
                self.tokenizer = AutoTokenizer.from_pretrained(self.base_model_id)
            self.tokenizer.pad_token_id = 0
            self.tokenizer.padding_side = "left"

        except Exception as e:
            logging.error(f"Error in downloading models: {e}")

    def tokenize_batch(
        self, data_points, add_eos_token=True, train_on_inputs=False, cutoff_len=512
    ) -> dict:
        """
        Tokenizes a batch of SQL related data points consisting of questions, context, and answers.

        Parameters:
            data_points (dict): A batch from the dataset containing 'question', 'context', and 'answer'.
            add_eos_token (bool): Whether to add an EOS token at the end of each tokenized sequence.
            cutoff_len (int): The maximum length for each tokenized sequence.

        Returns:
            dict: A dictionary containing tokenized 'input_ids', 'attention_mask', and 'labels'.
        """
        try:
            question = data_points["question"]
            context = data_points["context"]
            answer = data_points["answer"]
            if train_on_inputs:
                user_prompt = generate_prompt_sql(question, context)
                tokenized_user_prompt = self.tokenizer(
                    user_prompt,
                    truncation=True,
                    max_length=cutoff_len,
                    padding=False,
                    return_tensors=None,
                )
                user_prompt_len = len(tokenized_user_prompt["input_ids"])
                if add_eos_token:
                    user_prompt_len -= 1

            combined_text = generate_prompt_sql(question, context, answer)
            tokenized = self.tokenizer(
                combined_text,
                truncation=True,
                max_length=cutoff_len,
                padding=False,
                return_tensors=None,
            )
            if (
                tokenized["input_ids"][-1] != self.tokenizer.eos_token_id
                and add_eos_token
                and len(tokenized["input_ids"]) < cutoff_len
            ):
                tokenized["input_ids"].append(self.tokenizer.eos_token_id)
                tokenized["attention_mask"].append(1)
            tokenized["labels"] = tokenized["input_ids"].copy()
            if train_on_inputs:
                tokenized["labels"] = [-100] * user_prompt_len + tokenized["labels"][
                    user_prompt_len:
                ]

            return tokenized
        except Exception as e:
            logging.error(
                f"Error in batch tokenization: {e}, Line: {e.__traceback__.tb_lineno}"
            )
            raise e

    def prepare_data(self, data, val_set_size=100) -> Dataset:
        """Prepare training and validation datasets."""
        try:
            train_val_split = data["train"].train_test_split(
                test_size=val_set_size, shuffle=True, seed=42
            )
            train_data = train_val_split["train"].shuffle().map(self.tokenize_batch)
            val_data = train_val_split["test"].shuffle().map(self.tokenize_batch)
            return train_data, val_data
        except Exception as e:
            logging.error(
                f"Error in preparing data: {e}, Line: {e.__traceback__.tb_lineno}"
            )
            raise e

    def train_model(self, train_data, val_data, training_args):
        """
        Fine-tune the model with the given training and validation data.

        Parameters:
            train_data (Dataset): Training data.
            val_data (Optional[Dataset]): Validation data.
            training_args (TrainingArguments): Training configuration.
        """
        try:
            self.model = self.model.to(DEVICE)
            self.model = prepare_model(self.model)
            self.model = get_peft_model(self.model, LORA_CONFIG)
            trainer = Trainer(
                model=self.model,
                train_dataset=train_data,
                eval_dataset=val_data,
                args=training_args,
                data_collator=DataCollatorForSeq2Seq(
                    self.tokenizer,
                    pad_to_multiple_of=8,
                    return_tensors="pt",
                    padding=True,
                ),
            )
            self.model.config.use_cache = False
            trainer.train()
            self.model.save_pretrained(self.model_path)
        except Exception as e:
            logging.error(f"Error in model training: {e}")

    def finetune(self, data_path, training_args):
        """
        Execute the fine-tuning pipeline.

        Parameters:
            data_path (str): Path to the data for fine-tuning.
            training_args (TrainingArguments): Training configuration.
        """
        try:
            self.setup_models()
            data = load_dataset(data_path)
            train_data, val_data = self.prepare_data(data)
            self.train_model(train_data, val_data, training_args)
        except KeyboardInterrupt:
            print("Interrupt received, saving model...")
            self.model.save_pretrained(f"{self.model_path}_interrupted")
            print(f"Model saved to {self.model_path}_interrupted")
            sys.exit(0)
        except Exception as e:
            logging.error(f"Error in fintuning: {e}")


The `lets_finetune` function serves as the orchestrator for the fine-tuning process, providing a high-level interface to initiate training with a set of defined parameters. It allows users to specify the device for training, the model to fine-tune, batch size, warm-up steps, learning rate, and the maximum number of steps to take during training. By configuring these parameters, users can tailor the fine-tuning process to fit the specific needs of their dataset and computational resources.

Upon invocation, the function sets up the fine-tuning environment, initializes the `FineTuner` class with the selected model and device, and defines the training arguments that control the behavior of the training loop. These arguments include strategies for saving checkpoints, evaluation frequency, learning rate scheduling, precision training settings, and more. With bf16 precision and efficient batching, the function is geared towards achieving a balance between training speed and memory usage.

Once the setup is complete, the `finetuner.finetune` method is called to begin the actual fine-tuning task using the provided dataset. If any errors occur during this process, they are logged for troubleshooting. This function is a crucial component of the fine-tuning workflow, encapsulating the complexity of the training setup and execution.


In [5]:
def lets_finetune(device=DEVICE, model=BASE_MODEL, per_device_train_batch_size=2, warmup_steps=100, learning_rate=3e-4, max_steps=200):
    if device != torch.device("cpu"):
        print(f"Finetuning on device: {ipex.xpu.get_device_name()}")
    print(f"Using model: {model}")
    print(f"per device batch size: {per_device_train_batch_size}")
    print(f"warmup steps: {warmup_steps}")
    print(f"learning rate: {learning_rate}")
    print(f"max steps: {max_steps}")
    try:
        finetuner = FineTuner(
            base_model_id=model, model_path=MODEL_PATH, device=device
        )
        training_args = TrainingArguments(
            per_device_train_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=2,
            warmup_steps=warmup_steps,
            save_steps=max_steps // 4,
            save_strategy="steps",
            eval_steps=max_steps // 4,
            evaluation_strategy="steps",
            max_steps=max_steps,
            learning_rate=learning_rate,
            # num_train_epochs=2,
            max_grad_norm=0.3,
            bf16=True,
            lr_scheduler_type="cosine",
            load_best_model_at_end=True,
            ddp_find_unused_parameters=False,
            group_by_length=True,
            save_total_limit=3,
            logging_steps=max_steps // 10,
            optim="adamw_hf",
            output_dir=ADAPTER_PATH,
            logging_dir="./logs",
            report_to= [],
        )
        finetuner.finetune(DATA_PATH, training_args)
    except Exception as e:
        logging.error(f"Error occurred: {e}")

## Fine-Tuning the Model
Now it's time to actually fine-tune the model. The `lets_finetune` function below takes care of this. It initializes a FineTuner object with the configurations you've set or left as default.

### What Does It Do?
- Initializes the FineTuner object with the base model and other configurations.
- Sets up training arguments like batch size, learning rate, evaluation steps, etc.
- Starts the fine-tuning process using the data and configurations provided.

In [6]:
lets_finetune()

Finetuning on device: Intel(R) Arc(TM) A770M Graphics
Using model: openlm-research/open_llama_3b_v2
per device batch size: 2
warmup steps: 100
learning rate: 0.0003
max steps: 200


Map:   0%|          | 0/78477 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'loss': 2.0321, 'learning_rate': 5.9999999999999995e-05, 'epoch': 0.0}
{'loss': 1.0263, 'learning_rate': 0.00011999999999999999, 'epoch': 0.0}
{'eval_loss': 0.682393491268158, 'eval_runtime': 6.5451, 'eval_samples_per_second': 15.279, 'eval_steps_per_second': 1.986, 'epoch': 0.0}
{'loss': 0.6114, 'learning_rate': 0.00017999999999999998, 'epoch': 0.0}
{'loss': 0.5986, 'learning_rate': 0.00023999999999999998, 'epoch': 0.0}
{'loss': 0.4158, 'learning_rate': 0.0003, 'epoch': 0.01}
{'eval_loss': 0.6204714179039001, 'eval_runtime': 6.7285, 'eval_samples_per_second': 14.862, 'eval_steps_per_second': 1.932, 'epoch': 0.01}
{'loss': 0.6238, 'learning_rate': 0.0002713525491562421, 'epoch': 0.01}
{'loss': 0.4599, 'learning_rate': 0.0001963525491562421, 'epoch': 0.01}
{'eval_loss': 0.5696825981140137, 'eval_runtime': 6.7261, 'eval_samples_per_second': 14.867, 'eval_steps_per_second': 1.933, 'epoch': 0.01}
{'loss': 0.5081, 'learning_rate': 0.0001036474508437579, 'epoch': 0.01}
{'loss': 0.5237, 'lea

### Test Your Fine-Tuned Language Model

Congratulations on successfully fine-tuning your Language Model for Text-to-SQL tasks! It's now time to put your model to the test. You have two models at your disposal: the base model, which is the original pre-trained model without fine-tuning, and the fine-tuned model, which has been optimized with LoRA for your specific Text-to-SQL tasks.


### TextToSQLGenerator: Generating SQL Queries from Text Prompts

The `TextToSQLGenerator` class is a pivotal component of our Text-to-SQL translation tool. It encapsulates the logic for generating SQL queries from natural language prompts, leveraging the power of large language models. Here's what you need to know about its implementation:

#### Initialization and Configuration:

Upon initialization, the `TextToSQLGenerator` class allows the selection of the underlying model. By setting the `use_lora` parameter to `True`, you can enable the fine-tuned LoRA model for inference. Otherwise, the base pre-trained model is used by default.

The constructor of the class is designed to be robust and user-friendly. It automatically selects the correct tokenizer based on the model ID, with a special case for the 'llama' models. The model is then loaded with options optimized for performance on CPUs, such as `low_cpu_mem_usage` and `load_in_4bit`, which are beneficial for environments with memory constraints.

For LoRA models, the constructor also takes care of loading the LoRA-specific checkpoints, ensuring that the fine-tuned parameters are utilized during inference.

#### Generating SQL Queries:

The `generate` method is where the actual translation occurs. Given a text prompt, the method encodes the prompt using the tokenizer, ensuring that it fits within the model's maximum length constraints. It then performs inference to generate the SQL query, carefully managing the GPU or CPU memory and leveraging half-precision computation when available for faster inference.

The method is equipped with parameters like `temperature` and `num_beams` to control the creativity and quality of the generated queries, ensuring they are both diverse and close to the expected outputs.

#### Usage:

With `TextToSQLGenerator`, users can easily compare the performance of the base model and the LoRA-enhanced model by instantiating two objects of the class with different `use_lora` settings. This facilitates a side-by-side comparison of the generated SQL queries, demonstrating the effectiveness of the fine-tuning process.


In [ ]:
class TextToSQLGenerator:
    """Handles SQL query generation for a given text prompt."""

    def __init__(self, use_lora=False):
        """
        Initialize the InferenceModel class.
        Parameters:
            use_lora (bool, optional): Whether to use LoRA model. Defaults to False.
        """
        try:
            # Choose the appropriate tokenizer based on the model name
            if 'llama' in self.base_model_id.lower():
                self.tokenizer = LlamaTokenizer.from_pretrained(self.base_model_id)
            else:
                self.tokenizer = AutoTokenizer.from_pretrained(self.base_model_id)
            self.model = AutoModelForCausalLM.from_pretrained(
                BASE_MODEL,
                low_cpu_mem_usage=True,
                load_in_4bit=True,
                optimize_model=False,
                use_cache=True,
            )
            if use_lora:
                self.model = PeftModel.from_pretrained(self.model, LORA_CHECKPOINT)
        except Exception as e:
            logging.error(f"Exception occurred during model initialization: {e}")
            raise
            
        self.model.to(DEVICE)
        self.max_length = 512
        self.tokenizer.pad_token_id = 0
        self.tokenizer.padding_side = "left"

    def generate(self, prompt, **kwargs):
        """Generates an SQL query based on the given prompt.
        Parameters:
            prompt (str): The SQL prompt.
        Returns:
            str: The generated SQL query.
        """
        try:
            encoded_prompt = self.tokenizer(
                prompt,
                truncation=True,
                max_length=self.max_length,
                padding=False,
                return_tensors="pt",
            ).input_ids.to(DEVICE)
            with torch.no_grad():
                with torch.xpu.amp.autocast():
                    outputs = self.model.generate(
                        input_ids=encoded_prompt,
                        do_sample=False,
                        max_length=self.max_length,
                        temperature=0.3,
                        num_beams=5,
                        repetition_penalty=1.2,
                    )
            generated = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return generated
        except Exception as e:
            logging.error(f"Exception occurred during query generation: {e}")
            raise




#### Steps to Follow:
1. **Select or Input a Natural Language Query**: Start with a natural language question or prompt that you want to translate into an SQL query. You can either choose from the preloaded sample data or input your own.

2. **Generate the SQL Query Using the Base Model**: Use the base model to generate an SQL query based on your natural language prompt. This will serve as a baseline for comparison.

3. **Generate the SQL Query Using the Fine-Tuned Model**: Next, generate the SQL query using the fine-tuned model. Observe the differences and improvements in the SQL query generated by the fine-tuned model.

4. **Compare the Outputs**: Look at the SQL queries generated by both models. Does the fine-tuned model capture the intent of the natural language prompt more accurately? Is the SQL query syntactically correct and optimized?

5. **Iterate and Refine**: Fine-tuning is an iterative process. If the outputs aren't as expected, consider revising the training parameters or dataset and fine-tune again.

#### Try with Sample Data:
The following code snippet automatically processes a few examples from the loaded dataset. Observe how the base model and the fine-tuned model perform on the same prompts.


In [ ]:
TEST_DATA = "./test_data/sample_test_data.json"
base_model = TextToSQLGenerator()
finetuned_model = TextToSQLGenerator(use_lora=True)
sample_data = load_dataset("json", data_files=TEST_DATA)["train"]
for row in sample_data:
    try:
        prompt = generate_prompt_sql(row["question"], context=row["context"])
        print("Using base model...")
        output = base_model.generate(prompt)
        print(f"\n\tbot response: {output}\n")

        print("Using finetuned model...")
        output = finetuned_model.generate(prompt)
        print(f"\n\tbot response: {output}\n")
    except Exception as e:
        logging.error(f"Exception occurred during sample processing: {e}")

In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json

import ipywidgets as widgets
from IPython.display import display, clear_output
from datasets import load_dataset

def format_and_print_json_objects(file_path):
    with open(file_path, 'r') as file:
        json_object_str = ''
        json_objects = []

        for line in file:
            json_object_str += line.strip()
            if line.strip() == '}':
                try:
                    json_object = json.loads(json_object_str)
                    json_objects.append(json_object)
                    json_object_str = ''
                except json.JSONDecodeError as e:
                    logging.error("JSON Decode Error:", e)
                    json_object_str = ''
                    continue

    # Now we have our list of JSON objects, let's format and print them
    for entry in json_objects:
        question = entry['question']
        context = entry['context']
        print(f"question: {question}\ncontext: {context}\n\n")

format_and_print_json_objects(TEST_DATA)

custom_query_input = widgets.Textarea(
    value="",
    placeholder="Type your natural language query here...",
    description="Question:",
    layout=widgets.Layout(width="95%", height="100px"),
    style={"description_width": "initial"}
)

custom_context_input = widgets.Textarea(
    value="",
    placeholder="Type your SQL schema here...",
    description="Context:",
    layout=widgets.Layout(width="95%", height="100px"),
    style={"description_width": "initial"}
)

def generate_custom_query(b):
    query = custom_query_input.value
    context = custom_context_input.value
    prompt = generate_prompt_sql(query, context=context)
    print("Using base model...")
    base_output = base_model.generate(prompt)  # base_model should be defined and loaded
    print(f"\n\tBase model SQL: {base_output}\n")
    print("Using fine-tuned model...")
    finetuned_output = finetuned_model.generate(prompt)  # finetuned_model should be defined and loaded
    print(f"\n\tFine-tuned model SQL: {finetuned_output}\n")

generate_custom_query_btn = widgets.Button(
    description="Generate SQL from Custom Query",
    button_style="primary",
)
generate_custom_query_btn.on_click(generate_custom_query)
output_area = widgets.Output()

ui = widgets.VBox([
    custom_query_input,
    custom_context_input,
    generate_custom_query_btn,
    output_area
])

display(ui)

question: What was the score of the game wehre the deportes savio were the home team?
context: CREATE TABLE table_name_78 (score VARCHAR, home VARCHAR)


question: In which championship did the winner have a score of −9 (66-69-73-71=279)?
context: CREATE TABLE table_name_83 (championship VARCHAR, winning_score VARCHAR)


question: During the championship where the winning score was −9 (66-69-73-71=279)?, who was the runner-up?
context: CREATE TABLE table_name_32 (runner_s__up VARCHAR, winning_score VARCHAR)


question: What was the winning score in the year 2002?
context: CREATE TABLE table_name_28 (winning_score VARCHAR, year VARCHAR)


question: What is the fewest goals of CD Alcoyano with more than 25 points?
context: CREATE TABLE table_name_57 (goals_against INTEGER, club VARCHAR, points VARCHAR)


question: What is the fewest number of wins that had fewer than 7 draws and more than 30 played?
context: CREATE TABLE table_name_26 (wins INTEGER, draws VARCHAR, played VARCHAR)


quest